In [7]:
import pickle
from tqdm.notebook import tqdm
from essay_evaluation.classifier import Classifier
from essay_evaluation.corpus import read_flip_texts, read_csv
from essay_evaluation.pipeline import Pipeline

## 1. Set paths

In [8]:
dataset_path = '/home/simon/Downloads/dataset.csv'
model_regressor_path = '/home/simon/Downloads/model_reg.pkl'
model_classifier_path = '/home/simon/Downloads/model_clf.pkl'
test_size = 98

## 2. Get scores for all documents
We first calculate the essay's score once, so we can compare how the essay improves/worsens when removing sentences. 

In [9]:
with open(model_regressor_path, 'rb') as fh:
    model_regressor = pickle.load(fh)
with open(model_classifier_path, 'rb') as fh:
    model_classifier = pickle.load(fh)
    
pipeline = Pipeline().lexical_variation_taaled().get_pipe()
# it would be nice if the classifier would be part of our Pipeline class. But it's not that simple because it has to 
# be the _last_ component
pipeline.add_pipe(Classifier(model_regressor, 'score_regression'), name=Classifier.name + 'reg', last=True)
pipeline.add_pipe(Classifier(model_classifier, 'score_classification'), name=Classifier.name + 'clf', last=True)

texts, _= read_csv(dataset_path)
texts = texts[:test_size]
docs = list(tqdm(pipeline.pipe(texts[:test_size]), total=len(texts)))

## 3. Modify the essays
We generate alternative essays by leaving out one sentence after another

In [10]:
def generate_alternatives(doc):
    sents = list(doc.sents)
    for i, _ in enumerate(sents):
        one_removed = sents[:i] + sents[i+1:]
        yield ' '.join([s.text for s in one_removed])

In [11]:
result = []
result_feature_matrix = []
for essay_i, doc in enumerate(tqdm(docs)):
    alt_texts = generate_alternatives(doc)
    for sent_i, alt_text in enumerate(tqdm(alt_texts, total=len(list(doc.sents)))):
        doc_alt = pipeline(alt_text)
        
        result.append({**{
            '#Essay': essay_i,
            '#Sentence': sent_i,
            'Sentence Grade Regression': doc_alt._.score_regression,
            'Sentence Grade Classification': doc_alt._.score_classification,
            'Essay Grade Regression': doc._.score_regression,
            'Essay Grade Classification': doc._.score_classification,
            'Text': doc_alt.text,
            
        },**doc_alt._.features})
        
        result_feature_matrix.append(list(doc_alt._.features.values()))

In [12]:
import csv
with open('/home/simon/Downloads/result_sentences.csv', 'w') as fh:
    writer = csv.DictWriter(fh, fieldnames=result[0].keys())
    writer.writeheader()
    writer.writerows(result)

import numpy as np
np.save('/home/simon/Downloads/result_feature_matrix.npy', np.array(result_feature_matrix))